### Wstęp do Uczenia Maszynowego 
##### Laboratorium 8

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

sns.set(style="darkgrid")

### 1. Naive Bayes

In [34]:
X = pd.read_csv("X.csv")
y = pd.read_csv("y.csv")

### Zadanie 1
-----
Przygotuj dane w następujący sposób:

a) Ze zbioru `X` wybierz tylko zmienne `"education", "workclass", "sex", "hours-per-week"`.

b) Dokonaj transformacji dla zmiennej `"education"`

    1, education in ['Masters', 'some-college', 'Bachelors', 'Doctorate']
    0, w przeciwnym przypadku

c) Dokonaj transformacji dla zmiennej `"workclass"`

    1, workclass == 'Private'
    0, w przeciwnym przypadku

d) Dokonaj transformacji dla zmiennej `"sex"`

    1, sex == 'Male
    0, w przeciwnym przypadku

e) Dokonaj transformacji dla zmiennej `"hours-per-week"`

    1, hours-per-week <= 40 
    0, w przeciwnym przypadku

In [35]:
X = X[["education", "workclass", "sex", "hours-per-week"]]

In [36]:
X[["education"]] = (X[["education"]] == "Bachelors") | (X[["education"]] == "Masters") | (X[["education"]] == "Doctorate") | (X[["education"]] == "some-college")

In [37]:
X[["education"]] = X[["education"]].astype(int)

In [38]:
X[["workclass"]] = (X[["workclass"]] == "Private")

In [39]:
X[["workclass"]] = X[["workclass"]].astype(int)

In [40]:
X[["sex"]] = (X[["sex"]] == "Male")

In [41]:
X[["sex"]] = X[["sex"]].astype(int)

In [42]:
X[["hours-per-week"]] = X[["hours-per-week"]] <= 40

In [43]:
X[["hours-per-week"]] = X[["hours-per-week"]].astype(int) 

In [44]:
X

,education,workclass,sex,hours-per-week
0,1,0,1,1
1,1,0,1,1
2,0,1,1,1
3,0,1,1,1
4,1,1,0,1
...,...,...,...,...
48837,1,1,0,1
48838,0,0,1,1
48839,1,1,1,0
48840,1,1,1,1


In [8]:
y.value_counts()

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

In [45]:
y = y.replace({"<=50K.": "<=50K", ">50K.": ">50K"})

In [46]:
y.value_counts()

income
<=50K     37155
>50K      11687
Name: count, dtype: int64

### Zadanie 2
-----
Wylicz prawdopodobieństwa dla poszczególnych zmiennych pod warunkiem `Y`. Oblicz przynależność do klasy dla pierwszej obserwacji ze zbioru testowego na postawie wyliczonych prawdopodobieństw.

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2011)

In [49]:
print('<=50K:', np.mean(y_train == '<=50K'))
print('>50K:', np.mean(y_train == '>50K'))

<=50K: 0.7624958411179075
>50K: 0.2375041588820925


In [50]:
all = pd.concat([X_train, y_train], axis=1)

In [51]:
all

,education,workclass,sex,hours-per-week,income
25542,0,0,1,1,<=50K
20604,0,1,0,1,<=50K
38621,1,0,1,1,<=50K
23306,0,1,1,1,<=50K
6405,0,1,0,1,<=50K
...,...,...,...,...,...
26263,0,1,0,1,<=50K
5691,1,1,1,1,<=50K
40,0,1,1,0,<=50K
4429,0,1,1,1,<=50K


In [ ]:
round(all.groupby(by="income")['education'].value_counts(normalize=True).unstack(), 4)

education,0,1
income,,
<=50K,0.8363,0.1637
>50K,0.5539,0.4461


In [54]:
round(all.groupby(by="income")['sex'].value_counts(normalize=True).unstack(), 4)

sex,0,1
income,,
<=50K,0.3879,0.6121
>50K,0.1526,0.8474


In [55]:
round(all.groupby(by="income")['workclass'].value_counts(normalize=True).unstack(), 4)

workclass,0,1
income,,
<=50K,0.2878,0.7122
>50K,0.3685,0.6315


In [56]:
round(all.groupby(by="income")['hours-per-week'].value_counts(normalize=True).unstack(), 4)

hours-per-week,0,1
income,,
<=50K,0.2331,0.7669
>50K,0.4958,0.5042


In [59]:
X_test.iloc[0,:]

education         0
workclass         0
sex               1
hours-per-week    1
Name: 10172, dtype: int32

 y <= 50K

In [ ]:
0.8363 * 0.2878 * 0.6121 * 0.7669 * 0.7624958411179075

0.08889967675229524

y > 50K

In [60]:
0.5042 * 0.3685 * 0.8474 * 0.5539 * 0.2375041588820925

0.020712445429622195

### Zadanie 3
-----
Przygotuj model Naive Bayes używająć funkcji `BernouliiNB`. 
Jakie są dostępne inne warianty modelu Naive Bayes i czym się charakteryzują?

In [61]:
from sklearn.naive_bayes import BernoulliNB

In [62]:
NB = BernoulliNB()
NB.fit(X_train, y_train)

c:\Users\frane\anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BernoulliNB()

In [63]:
pred = NB.predict(X_test)

In [64]:
pred[0]

'<=50K'

In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       <=50K       0.79      0.97      0.87      7362
        >50K       0.66      0.20      0.30      2407

    accuracy                           0.78      9769
   macro avg       0.72      0.58      0.59      9769
weighted avg       0.75      0.78      0.73      9769



In [66]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[7114,  248],
       [1933,  474]], dtype=int64)

In [ ]:
from sklearn.metrics import roc_auc_score
pred_prob = NB.predict_proba(X_test)
roc_auc_score(y_test, pred_prob[:,1])

0.7491039954438781

### 2. kNN

In [97]:
df = pd.read_csv("SAheart.data", index_col='row.names').reset_index(drop=True)

### Zadanie 4
-----

a) Podziel losowo zbiór na część treningową i testową (7:3).

b) Przeskaluj zmienne w zbiorze treningowym i testowym za pomocą średniej i odchylenia standardowego danej zmiennej na zbiorze treningowym.

c) Dopasuj metodę k najbliższych sąsiadów dla $k = 5, 10, 20, 50, 80, 100, 150$ na zbiorze treningowym. Rozważ przynajmniej 3-krotną kroswalidację.

d) Sprawdź na zbiorze testowym współczynnik AUC dla wyliczonych $k$ z polecenia c).

e) Wybierz optymalne $k$ bazując na punkcie c) i d). Przygotuj model kNN z jednakową wagą dla wszystkich sąsiadów oraz z wagą uzależnioną od odległości między obserwacją a jej sąsiadem.

In [98]:
y = df.chd
X = df.drop(['chd'], axis = 1)

In [99]:
X = pd.get_dummies(X, drop_first=True, dtype=int)

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2011)

In [101]:
X_train.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,famhist_Present
225,143,5.04,4.86,23.59,58,24.69,18.72,42,0
279,118,0.12,3.26,12.26,55,22.65,0.00,16,0
392,178,0.95,4.75,21.06,49,23.74,24.69,61,0
76,134,0.00,5.90,30.84,49,29.16,0.00,55,0
181,152,0.00,6.06,41.05,51,40.34,0.00,51,1


In [102]:
from sklearn.preprocessing import StandardScaler

for col in X_train.columns:
    scaler = StandardScaler()
    X_train[col] = scaler.fit_transform(X_train[[col]])
    X_test[col] = scaler.transform(X_test[[col]])

In [103]:
X_train.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,famhist_Present
225,0.222007,0.313972,0.056981,-0.245566,0.514464,-0.326276,0.091433,-0.075534,-0.831324
279,-0.965215,-0.778831,-0.732078,-1.702061,0.204249,-0.820290,-0.707998,-1.882759,-0.831324
392,1.884118,-0.594476,0.002733,-0.570803,-0.416181,-0.556331,0.346380,1.245130,-0.831324
76,-0.205393,-0.805485,0.569869,0.686436,-0.416181,0.756195,-0.707998,0.828078,-0.831324
181,0.649407,-0.805485,0.648775,1.998952,-0.209371,3.463583,-0.707998,0.550044,1.202901


In [104]:
X_test.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,famhist_Present
246,-0.110415,-0.716639,-0.411523,-0.565661,1.031489,-0.905047,-0.707998,0.897587,1.202901
48,-1.155171,-0.805485,-1.383052,-1.861465,0.100844,-1.420855,0.956629,-1.882759,-0.831324
39,-0.110415,1.682198,0.525485,0.816274,2.272350,0.397793,0.271647,1.036604,1.202901
459,-1.440104,-0.139141,-1.555659,-1.320261,-1.346827,-1.440228,0.429654,0.828078,-0.831324
74,-0.490326,-0.716639,0.703023,0.109237,1.134894,0.441382,-0.233550,-0.631604,-0.831324


In [106]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

param_grid = {'n_neighbors': [5, 10, 20, 50, 80, 100, 150, 180]}

grid_search = GridSearchCV(knn, param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=2011), scoring='roc_auc')

In [107]:
grid_search.fit(X_train, y_train)

c:\Users\frane\anaconda3\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\frane\anaconda3\envs\ML\lib\site-packages\sklearn\metrics\_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
  File "c:\Users\frane\anaconda3\envs\ML\lib\site-packages\sklearn\metrics\_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\frane\anaconda3\envs\ML\lib\site-packages\sklearn\utils\_response.py", line 73, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
  File "c:\Users\frane\anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py", line 1940, in _check_response_method
    raise AttributeError(
AttributeError: KNeighborsClassifier has none of the following attr

GridSearchCV(cv=KFold(n_splits=5, random_state=2011, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 10, 20, 50, 80, 100, 150, 180]},
             scoring='roc_auc')

In [108]:
grid_search.best_params_

{'n_neighbors': 50}

In [ ]:
knn = KNeighborsClassifier(n_neighbors=50)

In [110]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=50)

In [111]:
print(classification_report(y_test, knn.predict(X_test)))

              precision    recall  f1-score   support

           0       0.79      0.95      0.86        97
           1       0.77      0.40      0.53        42

    accuracy                           0.78       139
   macro avg       0.78      0.68      0.70       139
weighted avg       0.78      0.78      0.76       139

